In [1]:
import requests
import json

In [2]:
API_KEY = 'V7lmZgPn7NGdEYSp9DGA8l1AsK5zRy8I'
#specifies the subject of the article. this one is for gun control
SUBJECT_ID = '4941756'
#the subject must match the article with a least a score of the min_score
MIN_SCORE = '0.7'
#time range
DAYS_BACK = '120' 
NUM_ARTICLES = '20'

##orberBy=magScore

In [8]:
r = requests.get('http://api.relegence.com/trending/4941756?lastActivityDaysBack=0&creationDaysBack=\
                    0&minScore=0.7&count=20&docsPerTopic=5&orberBy=magScore&apikey=' + API_KEY)

#### About data
results json:
'_id' = Story ID. A cluster of articles specifying the same story. To get all the stories from a Story ID you call the same API just with the Story ID instead of the Subject ID.

[u'alphaDocs',
 u'relatedEntity',
 u'facebookShares',
 u'title',
 u'lastTopicActivityTime',
 u'facebookLikes',
 u'topicCreationTime',
 u'heat',
 u'relatedSubject',
 u'facebookComments',
 u'avgAccessTime',
 u'twitterRetweets',
 u'magScore',
 u'numDocs',
 u'_id',
 u'numSources']

In [7]:
r2 = requests.get('http://api.relegence.com/trending/topic/documents/91485332?apikey=V7lmZgPn7NGdEYSp9DGA8l1AsK5zRy8I&onlyOnO=false&count=100')

In [8]:
data2 = r2.json()

In [10]:
data2[0]['guid']

u'http://wrbl.com/ap/obama-initiative-on-gun-control-shows-limits-of-acting-alone/'

### Whole content dataset

In [8]:
#construct request string based on 
# http://stage.api.relegence.com/stories/91485332?apikey=V7lmZgPn7NGdEYSp9DGA8l1AsK5zRy8I&withDocs=true
#beg + subjectid+ '?apikey=' + apikey + '&withDocs=true'
SUBJECT_ID = '91485332' #other subject_ids don't work
count = 30
content_request = 'http://stage.api.relegence.com/stories/' + SUBJECT_ID + '?apikey=' + API_KEY + '&withDocs=true'

In [9]:
test = 'http://stage.api.relegence.com/stories/91485332?apikey=V7lmZgPn7NGdEYSp9DGA8l1AsK5zRy8I&withDocs=true'

In [10]:
test == content_request

True

In [6]:
content_request

'http://stage.api.relegence.com/stories/91485332?apikey=V7lmZgPn7NGdEYSp9DGA8l1AsK5zRy8I&withDocs=true&count=30'

In [7]:
test

'http://stage.api.relegence.com/stories/91485332?apikey=V7lmZgPn7NGdEYSp9DGA8l1AsK5zRy8I&withDocs=true'

In [11]:
cont = requests.get(content_request)

In [12]:
cont = cont.json()

In [13]:
cont.keys()

[u'lastUpdate',
 u'facebookShares',
 u'title',
 u'facebookLikes',
 u'numOriginalDocs',
 u'creationTime',
 u'socialDistPercentage',
 u'relatedTags',
 u'facebookComments',
 u'avgAccessTime',
 u'twitterRetweets',
 u'magScore',
 u'articles',
 u'numTotalDocs',
 u'id']

In [14]:
len(cont['articles'])

10

In [15]:
#i think this means that this topic should have 1536 total documents but above says it only has 10
cont['numTotalDocs']

1536

In [16]:
cont['articles'][0].keys()

[u'lang',
 u'content',
 u'updated',
 u'title',
 u'media',
 u'published',
 u'snippet',
 u'source',
 u'link',
 u'provider',
 u'crawled_ts',
 u'license_id',
 u'guid',
 u'id']

In [17]:
cont['articles'][0]['source']

{u'id': u'O0RetC'}

In [133]:
cont['articles'][0]

{u'content': u'WASHINGTON (AP) - President Barack Obama\'s plan to strengthen controls on guns in the U.S. is meeting swift resistance from gun rights groups, Republicans and even a few Democrats who say it\'s up to Congress to enact new policies on firearms. Yet the overall effect on gun violence could prove to be relatively small. \nSome questions and answers about Obama\'s presidential actions on gun control: \nWHAT ACTION IS OBAMA TAKING? \nObama announced a 10-point plan to try to keep guns from people who shouldn\'t have them. The centerpiece is new federal guidance that seeks to clarify who is "in the business" of selling firearms and has to get a federal license. \nLicensed dealers must run background checks on prospective buyers, but private sellers don\'t. Obama is aiming to narrow that loophole so that more firearms sold at gun shows, flea markets or online are subject to background checks. \nOther steps include 230 new examiners the FBI will hire to process background check

In [20]:
#this takes the json response from the api and returns article content, title, and author

def get_sentences(content):
    articles = []
    for article in cont['articles']:
        art_dic = {}
        art_dic['content'] = article['content']
        art_dic['title'] = article['title']
        art_dic['source'] = article['source']
        articles.append(art_dic)
    return articles

In [21]:
articles = get_sentences(cont)

In [27]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [31]:
for word in test:
    print wordnet_lemmatizer.lemmatize(word)

Questions
and
answer
about
Obama's
executive
plan
on
gun


In [32]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
for word in test:
    print porter_stemmer.stem(word)

Question
and
answer
about
Obama'
execut
plan
on
gun


In [33]:
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer('english')
for word in test:
    print snowball_stemmer.stem(word)

question
and
answer
about
obama
execut
plan
on
gun


In [129]:
#for all sentences, lemmatize, don't lowercase
from nltk.stem import SnowballStemmer
import nltk.data

def stemming(sentence):
    snowball_stemmer = SnowballStemmer('english')
    new_sentence = []
    for word in sentence:
        new_sentence.append(snowball_stemmer.stem(word))
    return new_sentence

#just put title as part of article content
def cleaned_articles(articles):
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    articles_cleaned = []
    for r in articles:
        sentences = []
        sentences.append(stemming(nltk.word_tokenize(r['title'])))
        content_sentences = sent_detector.tokenize(r['content'])
        for sentence in content_sentences:
            sentences.append(stemming(nltk.word_tokenize(sentence)))
        articles_cleaned.append(sentences)
    return articles_cleaned

In [130]:
cleaned = cleaned_articles(articles)

In [122]:
tk = nltk.word_tokenize(csl[0][0])

In [123]:
# nltk.download()
nltk.pos_tag(tk)

[(u'WASHINGTON', 'NNP'),
 (u'(', '('),
 (u'AP', 'NNP'),
 (u')', ')'),
 (u'-', ':'),
 (u'President', 'NNP'),
 (u'Barack', 'NNP'),
 (u'Obama', 'NNP'),
 (u"'s", 'POS'),
 (u'plan', 'NN'),
 (u'to', 'TO'),
 (u'strengthen', 'VB'),
 (u'controls', 'NNS'),
 (u'on', 'IN'),
 (u'guns', 'NNS'),
 (u'in', 'IN'),
 (u'the', 'DT'),
 (u'U.S.', 'NNP'),
 (u'is', 'VBZ'),
 (u'meeting', 'VBG'),
 (u'swift', 'JJ'),
 (u'resistance', 'NN'),
 (u'from', 'IN'),
 (u'gun', 'NN'),
 (u'rights', 'NNS'),
 (u'groups', 'NNS'),
 (u',', ','),
 (u'Republicans', 'NNPS'),
 (u'and', 'CC'),
 (u'even', 'RB'),
 (u'a', 'DT'),
 (u'few', 'JJ'),
 (u'Democrats', 'NNPS'),
 (u'who', 'WP'),
 (u'say', 'VBP'),
 (u'it', 'PRP'),
 (u"'s", 'VBZ'),
 (u'up', 'RP'),
 (u'to', 'TO'),
 (u'Congress', 'NNP'),
 (u'to', 'TO'),
 (u'enact', 'VB'),
 (u'new', 'JJ'),
 (u'policies', 'NNS'),
 (u'on', 'IN'),
 (u'firearms', 'NNS'),
 (u'.', '.')]

### now do something with this

In [132]:
#save to pickle and explore in new file
#we have 206 sentences only
import pickle
pickle.dump(cleaned, open("cleaned_sentences_initial.p", "wb"))

#### do some information gain stuff?